# VAR-LiNGAM

LiNGAMにVAR（Vector Autoregression）モデルを組み合わせて時系列に拡張したもの。LiNGAMと同様な仮定を置いている。

:::{card} 仮定

1. 誤差変数は非ガウス分布
2. 線形の構造方程式モデル
3. 非巡回性
4. 隠れた共通原因がない

:::



通常の VAR は「過去が未来を予測できるか」という グレンジャー因果性（Granger causality） に基づくが、

- 同時刻内の因果方向は識別できない  
- 構造的（介入的）因果解釈が弱い  

という制約がある。VAR-LiNGAM はこれに対し、

- 同時刻の因果構造（DAG）
- 時間遅れの因果効果

を同一モデル内で明示的に推定する。

## VARモデルについて

変数が$n$次元、ラグの次数が$p$だとする。（例えば時点$t$の観測値は $x_{1,t}, x_{2,t}, \dots, x_{n,t}$ となる。）

VAR($p$) では、各変数 $x_{i,t}$ は、すべての変数の過去 $p$ 期分の線形結合で表される。

:::{card} VAR($p$)

$$
x_{i,t}
=
\sum_{k=1}^{p}
\sum_{j=1}^{n}
a_{ij}^{(k)} \, x_{j,t-k}
+
\varepsilon_{i,t}
\quad
(i = 1,\dots,n)
$$

- $a_{ij}^{(k)}$：変数 $j$ の $k$ 期前」が「変数 $i$ の現在」に与える影響
- $\varepsilon_{i,t}$：変数 $i$ に対応する誤差項

:::


状態ベクトル $\mathbf{x}_t \in \mathbb{R}^n$ 、
ラグ $k$ の係数行列 $\mathbf{A}_k \in \mathbb{R}^{n \times n}$、
誤差ベクトル $\boldsymbol{\varepsilon}_t$ をそれぞれ

$$
\mathbf{x}_t
=
\begin{pmatrix}
x_{1,t} \\
x_{2,t} \\
\vdots \\
x_{n,t}
\end{pmatrix}
,\quad
\mathbf{A}_k
=
\begin{pmatrix}
a_{11}^{(k)} & a_{12}^{(k)} & \cdots & a_{1n}^{(k)} \\
a_{21}^{(k)} & a_{22}^{(k)} & \cdots & a_{2n}^{(k)} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n1}^{(k)} & a_{n2}^{(k)} & \cdots & a_{nn}^{(k)}
\end{pmatrix}
,\quad
\boldsymbol{\varepsilon}_t
=
\begin{pmatrix}
\varepsilon_{1,t} \\
\varepsilon_{2,t} \\
\vdots \\
\varepsilon_{n,t}
\end{pmatrix}
$$

とおくと、VAR($p$) は次のように表現できる。

:::{card} VAR($p$) （行列を使った表現）

$$
\mathbf{x}_t
=
\sum_{k=1}^{p}
\mathbf{A}_k \mathbf{x}_{t-k}
+
\boldsymbol{\varepsilon}_t
$$

:::


※切片を含めた表現もされる

$$
\mathbf{x}_t
=
\mathbf{c}
+
\sum_{k=1}^{p}
\mathbf{A}_k \mathbf{x}_{t-k}
+
\boldsymbol{\varepsilon}_t
$$


## VAR-LiNGAM

### モデル

$p$ 次元の時系列 $\mathbf{x}_t$ に対し、VAR-LiNGAM は以下を仮定する。

$$
\mathbf{x}_t
= \mathbf{B}_0 \mathbf{x}_t
+ \sum_{k=1}^{K} \mathbf{B}_k \mathbf{x}_{t-k}
+ \mathbf{e}_t
$$

- $\mathbf{B}_0$：同時（instantaneous）因果効果行列。DAG構造。
  - 有向非巡回グラフ（DAG）構造
- $\mathbf{B}_k$：ラグ $k$ における因果効果
- $\mathbf{e}_t$：成分間で独立な非ガウス分布に従う誤差

#### 構造 VAR 表現

$$
(\mathbf{I} - \mathbf{B}_0)\mathbf{x}_t
= \sum_{k=1}^{K} \mathbf{B}_k \mathbf{x}_{t-k}
+ \mathbf{e}_t
$$

$\mathbf{I} - \mathbf{B}_0$ は、変数の並び替えにより下三角行列にできる（＝因果順序が存在）ことが仮定される。

### アルゴリズム

VAR-LiNGAM は **2段階推定**で実装されることが多い。

#### Step 1: VAR 部分の推定

通常の VAR を推定する。

$$
\mathbf{x}_t
= \sum_{k=1}^{K} \mathbf{A}_k \mathbf{x}_{t-k}
+ \mathbf{r}_t
$$

- OLS 等で推定
- 残差 $\mathbf{r}_t$ を取得

#### Step 2: 残差に LiNGAM を適用

- $\mathbf{r}_t$ に **i.i.d. LiNGAM** を適用
- ICA（Independent Component Analysis）により
  - 変数の因果順序
  - 同時因果行列 $\mathbf{B}_0$

を推定する。

## 実装

LiNGAMパッケージにVARLiNGAMが実装されている

:::{note}

[VARLiNGAM — LiNGAM 1.11.0 documentation](https://lingam.readthedocs.io/en/latest/tutorial/var.html)

:::